### sql-cheatsheets
![sql cheat sheet page1](sql-page-1.png)

![sql cheat sheet page2](sql-page-2.png)

### Review Banking DB

[schema](banks/schema.sql)

[seed](banks/seed.sql)

### Query

-- who are the customers names paid through Wells Fargo?
```sql 
CREATE VIEW payment_customer_bank AS
(SELECT p.*, c.first_name, c.last_name, b.bank_name
FROM payments p
JOIN customer c ON p.customer_id = c.customer_id
JOIN banks b ON p.bank_routing_number = b.bank_routing_number)
WHERE bank_name = 'Wells Fargo';

SELECT * from payment_customer_bank
WHERE bank_name = 'Wells Fargo'
```



[Sakila DB](https://dev.mysql.com/doc/sakila/en/sakila-structure-tables.html)

-- Identify customers who rented the film 'PANIC CLUB'. first method by creating a giant table including everything, then filter.
```sql
SELECT c.customer_id, c.first_name, c.last_name, f.title
FROM customer c
JOIN rental r ON c.customer_id  = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE f.title = 'PANIC CLUB'
```

-- second method sub queries
```sql
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
WHERE c.customer_id IN (
    SELECT r.customer_id
    FROM rental r
    WHERE r.inventory_id IN (
        SELECT i.inventory_id
        FROM inventory i
        WHERE i.film_id IN (
            SELECT f.film_id
            FROM film f
            WHERE f.title = 'PANIC CLUB'
        )
    )
);
```


### Steps
1. Think about the tables getting involved. 
2. think about what kind cols
3. think about how to construct the query. 2 methods. 

--1. How many customers in total?
```sql
SELECT COUNT(*) AS total_customers_num
FROM customer

--2. What is the total amount of payment received?
SELECT SUM(amount) AS total_payment_amount
FROM payment;

--3. How many films in our inventory?

SELECT COUNT(*) AS total_films
FROM film

-- 4. What is the average rental rate for the films?
SELECT AVG(rental_rate) AS average_rental_rate
FROM film

--5. List the top 10 customers who have made the most number of rentals. (names and counts )
SELECT c.first_name, c.last_name, COUNT(r.rental_id) AS rental_count
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY c.customer_id
ORDER BY rental_count DESC
LIMIT 10

--6. What are the top 10 most rented films? (1. films, inventory, and rental. 2. count of film_id in rental table trough inventory table, film.title film_id )

SELECT f.film_id, f.title, COUNT(r.rental_id) AS rental_count
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id
ORDER BY rental_count DESC
LIMIT 10

--7. Calculate the total revenue for each store (1. store, staff, payment)
SELECT s.store_id, SUM(p.amount) AS total_revenue
FROM store s
JOIN staff st ON s.store_id = st.store_id
JOIN payment p ON st.staff_id = p.staff_id
GROUP BY s.store_id

-- from Hyun Bin, we agree it's the correct answer
SELECT store.store_id, SUM(payment.amount) FROM payment
INNER JOIN rental ON payment.rental_id = rental.rental_id
INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
INNER JOIN store ON store.store_id = inventory.store_id
GROUP BY store.store_id